In [26]:
from PyPDF2 import PdfReader
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [2]:
tempelate = """
{context}
Question: {question}
Helpful Answer:
"""

In [25]:
llm=LlamaCpp(
    model_path = r"F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",
    max_tokens = 512,
    CallbackManager = CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose= True,
    temperature = 0.4,
    stop = ['User :'],
    n_ctx=4096

)

c:\Users\Sunil\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\utils\utils.py:161: UserWarning: WARNING! CallbackManager is not default parameter.
                CallbackManager was transferred to model_kwargs.
                Please confirm that CallbackManager is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 23 key-value pairs and 201 tensors from F:\LLM\Tiny\tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.


llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = tinyllama_tinyllama-1.1b-chat-v1.0
llama_model_loader: - kv   2:                       llama.context_length u32              = 2048
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 2048
llama_model_loader: - kv   4:                          llama.block_count u32              = 22
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 5632
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 64
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   8:              llama.attention.head_count_kv u32              = 4
llama_model_loader: - kv   9:     llama.attention.layer_norm_rms_epsilon f32              = 0.000010
llam

In [4]:
# from PyPDF2 import PdfReader

# def extract_text_from_pdf(pdf_file):
#     text = ""
#     pdf_reader = PdfReader(pdf_file)
#     for page in pdf_reader.pages:
#         text += page.extract_text()
#     return text

# # Usage example
# pdf_path = r"F:\LLM\Vedantcv (3).pdf"
# extracted_text = extract_text_from_pdf(pdf_path)
# # print(extracted_text)

In [11]:
embeddings = HuggingFaceEmbeddings(model_name=r'F:\LLM\Sentence_Transformers_model', model_kwargs={'device':'cpu'})

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000,chunk_overlap=200)

New pdf loading !!!

In [16]:
from langchain.document_loaders import PyPDFLoader


loader = PyPDFLoader(r"F:\LLM\Vedantcv (3).pdf")
data = loader.load_and_split()

In [17]:
texts = text_splitter.split_documents(data) 

In [18]:
vdb = FAISS.from_documents(texts,embeddings)

In [21]:
vdb.save_local("resumevdb")

In [23]:
nvdb= FAISS.load_local("resumevdb",embeddings)

In [24]:
retriever= nvdb.as_retriever(serach_type="similarity",search_kwargs={"k":2},device="cuda")


In [28]:
# Create a question answering system based on information retrieval using the RetrievalQA class, which takes as input a neural language model, a chain type and a retriever (an object that allows you to retrieve the most relevant chunks of text for a query)
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [30]:
# Create an infinite loop to interact with the system, asking the user to enter a query or exit the program
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  # pass the query to the system and print the response
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     538.76 ms
llama_print_timings:      sample time =      91.66 ms /   176 runs   (    0.52 ms per token,  1920.08 tokens per second)
llama_print_timings: prompt eval time =   11656.22 ms /   308 tokens (   37.84 ms per token,    26.42 tokens per second)
llama_print_timings:        eval time =    9714.44 ms /   175 runs   (   55.51 ms per token,    18.01 tokens per second)
llama_print_timings:       total time =   21730.63 ms /   483 tokens


Answer:  vedant's passion is machine learning, and he's actively involved in hands-on projects to deepen his knowledge of the field. He's eagerly contributing creatively to cutting-edge engineering projects leveraging a multi-disciplinary approach. His experience includes working as a machine learning intern at nullclasses, data analytics intern at ibm, web-development intern at oasis infobyte, and flutter app development intern at logithon hackathon. He also holds certifications in advanced learning algorithms, supervised machine learning, and data analysis with python (ibm). His personal projects include disease prediction by extracting data from medical reports using llm and logistic regression, attendantance system using face recognition, sign language recognition, pedestrian detection and count system, sales analysis and prediction for multiple products.
Exiting


SystemExit: 

c:\Users\Sunil\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
